In [ ]:
import json
import numpy as np
import os
import pandas as pd
from pyspark.sql import types as st
from pyspark.sql import functions as sf
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

In [ ]:
config = [
    ("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.5-s_2.13"),
    ("spark.driver.memory", "8g"),
    ("spark.worker.memory", "8g"),
    ("spark.sql.session.timeZone", "UTC")
]
spark = SparkSession.builder.appName("preprocessing").config(conf=SparkConf().setAll(config)).getOrCreate()

In [ ]:
with open("../config.json") as f:
    config = json.load(f)

DATASET = config["dataset"]
DATA_PATH = "../data/01-ibm-transactions-for-aml"

RAW_DATA_PATH = os.path.join(DATA_PATH, "raw")
PREPROCESSED_DATA_PATH = os.path.join(DATA_PATH, "preprocessed")

In [ ]:
TIMESTAMP_FORMAT = "yyyy/MM/dd HH:mm"

INPUT_DATA_FILE = os.path.join(RAW_DATA_PATH, f"{DATASET}_Trans.csv")
INPUT_PATTERNS_FILE = os.path.join(RAW_DATA_PATH, f"{DATASET}_Patterns.txt")

STAGED_TRANS_LOCATION = os.path.join(PREPROCESSED_DATA_PATH, f"{DATASET}-transactions")
STAGED_DATA_FILE = os.path.join(PREPROCESSED_DATA_PATH, f"{DATASET}-staged-transactions.csv")
STAGED_PATTERNS_LOCATION = os.path.join(PREPROCESSED_DATA_PATH, f"{DATASET}-patterns")
STAGED_PATTERNS_CSV_LOCATION = os.path.join(STAGED_PATTERNS_LOCATION, f"{DATASET}-patterns.txt")
STAGED_CASES_DATA_LOCATION = os.path.join(STAGED_PATTERNS_LOCATION, f"{DATASET}-cases.parquet")

In [ ]:
def get_id(value):
    return f"id-{hash(value)}"

In [ ]:
os.makedirs(os.path.dirname(STAGED_DATA_FILE), exist_ok=True)

try:
    os.remove(STAGED_DATA_FILE)
except FileNotFoundError:
    pass

with open(INPUT_DATA_FILE) as input_file:
    cnt = 0
    lines = ""
    for line in input_file:
        cnt += 1
        if cnt == 1:
            continue
        line = line.strip()
        lines += f"{get_id(line)},{line}\n"
        if cnt % (1e6) == 0:
            with open(STAGED_DATA_FILE, "a") as output_file:
                output_file.write(lines)
                lines = ""
if lines:
    lines = lines.strip()
    with open(STAGED_DATA_FILE, "a") as output_file:
        output_file.write(lines)
        del lines

In [ ]:
os.makedirs(STAGED_PATTERNS_LOCATION, exist_ok=True)

try:
    os.remove(STAGED_PATTERNS_CSV_LOCATION)
except FileNotFoundError:
    pass

lines = ""
with open(INPUT_PATTERNS_FILE) as input_file:
    for line in input_file:
        line = line.strip()
        if line[:4].isnumeric():
            lines += f"{get_id(line)},{line}\n"
        else:
            lines += f"{line}\n"
lines = lines.strip()
with open(STAGED_PATTERNS_CSV_LOCATION, "a") as output_file:
    output_file.write(lines)
    del lines

In [ ]:
schema = st.StructType(
    [
        st.StructField("transaction_id", st.StringType(), False),
        st.StructField("timestamp", st.TimestampType(), False),
        st.StructField("source_bank", st.StringType(), False),
        st.StructField("source", st.StringType(), False),
        st.StructField("target_bank", st.StringType(), False),
        st.StructField("target", st.StringType(), False),
        st.StructField("received_amount", st.FloatType(), False),
        st.StructField("receiving_currency", st.StringType(), False),
        st.StructField("sent_amount", st.FloatType(), False),
        st.StructField("sending_currency", st.StringType(), False),
        st.StructField("format", st.StringType(), False),
        st.StructField("is_laundering", st.IntegerType(), False),
    ]
)
columns = [x.name for x in schema]

In [ ]:
with open(STAGED_PATTERNS_CSV_LOCATION, "r") as fl:
    patterns = fl.read()

cases = []
case_id = 0
for pattern in patterns.split("\n\n"):
    case_id += 1
    if not pattern.strip():
        continue
    pattern = pattern.split("\n")
    name = pattern.pop(0).split(" - ")[1]
    category, sub_category = name, name
    if ": " in name:
        category, sub_category = name.split(": ")
    pattern.pop()
    case = pd.DataFrame([x.split(",") for x in pattern], columns=columns)
    case.loc[:, "id"] = case_id
    case.loc[:, "type"] = category
    case.loc[:, "sub_type"] = sub_category
    cases.append(case)
cases = pd.concat(cases, ignore_index=True)
cases = spark.createDataFrame(cases)
cases = cases.withColumn("timestamp", sf.to_timestamp("timestamp", TIMESTAMP_FORMAT))
cases = cases.select(
    "transaction_id", "id", "type", "sub_type"
)

In [ ]:
CURRENCY_MAPPING = {
    "Australian Dollar": "aud",
    "Bitcoin": "btc",
    "Brazil Real": "brl",
    "Canadian Dollar": "cad",
    "Euro": "eur",
    "Mexican Peso": "mxn",
    "Ruble": "rub",
    "Rupee": "inr",
    "Saudi Riyal": "sar",
    "Shekel": "ils",
    "Swiss Franc": "chf",
    "UK Pound": "gbp",
    "US Dollar": "usd",
    "Yen": "jpy",
    "Yuan": "cny"
}

currency_code = sf.udf(lambda x: CURRENCY_MAPPING[x], st.StringType())

In [ ]:
data = spark.read.csv(STAGED_DATA_FILE, header=False, schema=schema, timestampFormat=TIMESTAMP_FORMAT)

group_by = [
    "timestamp",
    "source_bank",
    "source",
    "target_bank",
    "target",
    "receiving_currency",
    "sending_currency",
    "format",
]

data = data.groupby(group_by).agg(
    sf.first("transaction_id").alias("transaction_id"),
    sf.collect_set("transaction_id").alias("transaction_ids"),
    sf.sum("received_amount").alias("received_amount"), 
    sf.sum("sent_amount").alias("sent_amount"),
    sf.max("is_laundering").alias("is_laundering"),
)
data = data.withColumn(
    "source_currency", currency_code(sf.col("sending_currency"))
).withColumn(
    "target_currency", currency_code(sf.col("receiving_currency")),
)
data = data.join(cases, on="transaction_id", how="left").repartition(128, "transaction_id")
data = data.select(
    "transaction_id",
    "transaction_ids",
    "timestamp",
    sf.concat(sf.col("source"), sf.lit("-"), sf.col("source_currency")).alias("source"),
    sf.concat(sf.col("target"), sf.lit("-"), sf.col("target_currency")).alias("target"),
    "source_bank",
    "target_bank",
    "source_currency",
    "target_currency",
    sf.col("sent_amount").alias("source_amount"),
    sf.col("received_amount").alias("target_amount"),
    "format",
    "is_laundering",
).persist(StorageLevel.DISK_ONLY)
data.count()

In [ ]:
data.select(sf.explode("transaction_ids")).count() - data.count()

In [ ]:
cases_data = cases.join(
    data.withColumnRenamed("transaction_id", "x").drop(*cases.columns).select(
        sf.explode("transaction_ids").alias("transaction_id"), "*"
    ),
    on="transaction_id",
    how="left",
).drop("is_laundering", "transaction_id", "transaction_ids").withColumnRenamed("x", "transaction_id")
cases_data.toPandas().to_parquet(STAGED_CASES_DATA_LOCATION)
cases_data = pd.read_parquet(STAGED_CASES_DATA_LOCATION)

In [ ]:
data = data.drop("transaction_ids")

data.write.mode("overwrite").parquet(STAGED_TRANS_LOCATION)
data = spark.read.parquet(STAGED_TRANS_LOCATION)

os.remove(STAGED_DATA_FILE)

In [ ]:
data = data.withColumn("date", sf.to_date("timestamp"))
data = data.orderBy("timestamp")

In [ ]:
data = data.withColumn(
    "converted_amount",
    sf.when(sf.col("source_currency") == "usd", sf.col("source_amount"))
    .when(sf.col("target_currency") == "usd", sf.col("target_amount"))
    .otherwise(0)
)

unique_currencies = data.select(sf.col("source_currency").alias("currency")).union(
    data.select(sf.col("target_currency").alias("currency"))
).distinct().collect()
unique_currencies = [row.currency for row in unique_currencies]

dates = data.select("date").distinct().orderBy("date").collect()
dates = sorted([row.date for row in dates])

source_usd = data.filter(sf.col("source_currency") == "usd").filter(sf.col("target_currency") != "usd")\
    .withColumn("rate", sf.col("target_amount") / sf.col("source_amount"))

target_usd = data.filter(sf.col("target_currency") == "usd").filter(sf.col("source_currency") != "usd")\
    .withColumn("rate", sf.col("source_amount") / sf.col("target_amount"))

source_usd.cache()
target_usd.cache()

In [ ]:
curr_dict = {date: {curr: 1 if curr == 'usd' else 0 for curr in unique_currencies} for date in dates}

combined_df = source_usd.select(sf.col("date"), sf.col("target_currency").alias("currency"), sf.col("rate")) \
    .union(target_usd.select(sf.col("date"), sf.col("source_currency").alias("currency"), sf.col("rate")))

mean_rates = combined_df.groupBy("date", "currency").agg(sf.mean("rate").alias("mean_rate"))

rate_dict = {(row.date, row.currency): row.mean_rate for row in mean_rates.collect()}

for date in dates:
    for currency in unique_currencies:
        if curr_dict[date][currency] == 0:
            curr_dict[date][currency] = rate_dict.get((date, currency), 0)

In [ ]:
df = pd.DataFrame(curr_dict).T
df.sort_index(inplace=True)
df.replace(0, np.nan, inplace=True)
df.ffill(axis=0, inplace=True)
curr_dict = df.T.to_dict()
curr_dict = {str(key): value for key, value in curr_dict.items()}

In [ ]:
data = data.withColumn("date_str", sf.date_format(sf.col("date"), "yyyy-MM-dd"))

broadcast_curr_dict = spark.sparkContext.broadcast(curr_dict)

def get_rate(date, currency):
    if currency =='usd':
        return 1.0
    return broadcast_curr_dict.value.get(date, {}).get(currency, 0)
rate_udf = sf.udf(get_rate, st.DoubleType())

data = data.withColumn("rate", rate_udf(sf.col("date_str"), sf.col("source_currency")))
data = data.withColumn("converted_amount", sf.col("source_amount") / sf.col("rate"))

In [ ]:
data = data.drop("date","date_str", "rate")
data = data.withColumnRenamed("converted_amount", "amount")
data = data.withColumn("amount", sf.round(data.amount, 6))
data = data.withColumn("timestamp", sf.date_format(sf.col("timestamp"), "yyyy-MM-dd HH:mm"))

In [ ]:
data.write.mode("overwrite").parquet(STAGED_TRANS_LOCATION)
spark.stop()